In [1]:
import pandas as pd
from gpt_sender_async import GPT_API as gpt
import asyncio
from tqdm import tqdm
import json

In [2]:
df = pd.read_csv("data.csv", index_col=0)
df = df[df["Предложенная Точка отказа"].isna()]

In [3]:
df.head(2)

,Тема,Описание,Тип оборудования,Точка отказа,Серийный номер,Предложенная Точка отказа
index,,,,,,
0,Не работает блок питания,В течении недели перестал внезапно работать бл...,Ноутбук,Блок питания,C223100360,NaN
3,Повреждение матрицы,Прошу произвести диагностику и сориентировать ...,Ноутбук,Матрица,NaN,NaN


In [5]:
def get_prompt(tema, opisanie, oborudovanie, tochka_otkaza):
    return '''"""
Тема: {}
Описание: {}
Оборудование: {}
Точка отказа: {}
"""
Выше представлено обращение пользователя в тех поддержку с темой, описанием и проблемой.
Сгенерируй 10 разных непохожих обращений, опираясь на точку отказа и оборудование на котором произошел отказ.

Верни в формате json. 
Пример: {{1 : {{"Тема": str, "Описание": str}}}}
Где вместо str должны быть тема и описание проблемы и так для 10 обращений. Return only JSON.
'''.format(tema, opisanie, oborudovanie, tochka_otkaza)
    

In [7]:
async def get_responses(prompts):
    tasks = [gpt.get_response(prompt, json_output=True) 
             for prompt in prompts]
    responses = await asyncio.gather(*tasks)
    return responses


In [12]:
batch_size = 8
result = []
for i in tqdm(range(0, len(df), batch_size)):
    # Add the batch to the batches list.

    data = []
    for index, row in df[i:i+batch_size].iterrows():
        data.append([row["Тема"], row["Описание"], row["Тип оборудования"], row["Точка отказа"]])

    summaries = await get_responses([get_prompt(*row) for row in data])

    for row, summary in zip(data, summaries):

        if summary is None:
            continue
        
        result.append({"Тип оборудования": row[2], "Точка отказа": row[3], "Обращения": summary})

100%|██████████| 21/21 [03:08<00:00,  8.97s/it]


In [13]:
with open("new_data_gpt4o_v2.jsonl", 'w') as file:
    for entry in result:
        json_line = json.dumps(entry,ensure_ascii=False)  # Convert dictionary to JSON string
        file.write(json_line + '\n')   